In [35]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [36]:
# Read the CSV file into a DataFrame
df = pd.read_csv('condition_filled.csv')

In [37]:
data = df[["ID mosta", 'n_2012', 'n_2013', 'n_2014',
       'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
       'n_2022', 'n_2023', 'n_length', 'n_width', 'n_region', 'n_class',
       'n_years', 'n_material', 'n_type', 'b_change' ]]

data_lt = data[data["b_change"] == -1]
data_eq = data[data["b_change"] == 0]
data_gt = data[data["b_change"] == 1]

print(data_eq.shape)

# Take 1000 random rows from data_eq
random_indices = np.random.choice(data_eq.index, size=2000, replace=False)
random_data_eq = data_eq.loc[random_indices]



# Concatenate random_data_eq with data_gt
new_df = pd.concat([random_data_eq, data_gt], ignore_index=True)
print(new_df.shape)
new_df = pd.concat([new_df, data_gt], ignore_index=True)
print(new_df.shape)


predict = "n_2023"
droplist = ["ID mosta", 'b_change', predict]

X = new_df.drop(droplist, axis=1)
y = new_df[predict]

n_layers = X.shape[1]

(6405, 21)
(2993, 21)
(3986, 21)


In [38]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
from sklearn.preprocessing import MinMaxScaler

# Create a Min-Max scaler
scaler = MinMaxScaler()

# Fit the scaler to your training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [46]:

# Define the model architecture using the number of normalized features
model = keras.Sequential([
    keras.layers.Input(shape=(n_layers,)),  # Number of input features
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the normalized training data
model.fit(X_train_normalized, y_train, epochs=250, batch_size=8, validation_split=0.1, verbose=True)

# Evaluate the model on the normalized test data
loss = model.evaluate(X_test_normalized, y_test)
# print(f"Mean Squared Error on Test Data: {loss}")


# # Write results to text file
# with open('results.txt', 'a') as f:
#     f.write(f"Iteration {i}:\n")
#     f.write(f"Percentage of correct predictions for data_eq: {round(comparison_df['check'].mean()*100, 2)}%\n") 
#     f.write(f"Percentage of correct predictions for data_gt: {round(comparison_gt['check'].mean()*100, 2)}%\n") 
#     f.write("\n")  # Add a newline to separate results for different iterations

Epoch 1/250
359/359 [==============================] - 1s 1ms/step - loss: 1.9467 - val_loss: 0.6706
Epoch 2/250
359/359 [==============================] - 0s 1ms/step - loss: 0.6270 - val_loss: 0.6145
Epoch 3/250
359/359 [==============================] - 0s 1ms/step - loss: 0.5891 - val_loss: 0.6247
Epoch 4/250
359/359 [==============================] - 0s 1ms/step - loss: 0.5720 - val_loss: 0.6129
Epoch 5/250
359/359 [==============================] - 0s 1ms/step - loss: 0.5526 - val_loss: 0.5728
Epoch 6/250
359/359 [==============================] - 0s 1ms/step - loss: 0.5412 - val_loss: 0.5564
Epoch 7/250
359/359 [==============================] - 0s 1ms/step - loss: 0.5330 - val_loss: 0.6272
Epoch 8/250
359/359 [==============================] - 0s 1ms/step - loss: 0.5224 - val_loss: 0.5480
Epoch 9/250
359/359 [==============================] - 0s 1ms/step - loss: 0.4998 - val_loss: 0.5485
Epoch 10/250
359/359 [==============================] - 0s 1ms/step - loss: 0.4962 - val_lo

In [47]:
# data = df[["ID mosta", 'n_2012', 'n_2013', 'n_2014',
#        'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
#        'n_2022', 'n_2023', 'n_length', 'n_width', 'n_region', 'n_class',
#        'n_years', 'n_material', 'n_type', 'b_change' ]]

# data = data[data["b_change"] == 1]

# predict = "n_2023"
# droplist = ["ID mosta", 'b_change', predict]

X = data_gt.drop(droplist, axis=1)
y = data_gt[predict]
X_normalized = scaler.transform(X)

# Make predictions on the normalized test data
predictions = model.predict(X_normalized)

# Round the predicted values to zero decimal places and convert to integers
rounded_predictions = np.round(predictions).astype(int)

# Create a DataFrame to compare the actual 'n_2023' values with the model's predictions
comparison_df = pd.DataFrame({'Actual': y, 'Predicted': rounded_predictions.flatten()})


# Add a new column "check" to the comparison DataFrame
comparison_df['check'] = comparison_df['Actual'] == comparison_df['Predicted']

# Display the comparison DataFrame
print(comparison_df["check"].value_counts())
print(f"Percentage of correct predictions: {round(comparison_df['check'].mean()*100, 2)}%")

X = data_eq.drop(droplist, axis=1)
y = data_eq[predict]
X_normalized = scaler.transform(X)

# Make predictions on the normalized test data
predictions = model.predict(X_normalized)

# Round the predicted values to zero decimal places and convert to integers
rounded_predictions = np.round(predictions).astype(int)

# Create a DataFrame to compare the actual 'n_2023' values with the model's predictions
comparison_df = pd.DataFrame({'Actual': y, 'Predicted': rounded_predictions.flatten()})


# Add a new column "check" to the comparison DataFrame
comparison_df['check'] = comparison_df['Actual'] == comparison_df['Predicted']

# Display the comparison DataFrame
print(comparison_df["check"].value_counts())
print(f"Percentage of correct predictions: {round(comparison_df['check'].mean()*100, 2)}%")
       

32/32 [==============================] - 0s 1ms/step
check
True     668
False    325
Name: count, dtype: int64
Percentage of correct predictions: 67.27%
201/201 [==============================] - 0s 856us/step
check
True     4775
False    1630
Name: count, dtype: int64
Percentage of correct predictions: 74.55%


In [138]:
for i in range(800, 1201, 50):
    print(f"i = {i}")

i = 800
i = 850
i = 900
i = 950
i = 1000
i = 1050
i = 1100
i = 1150
i = 1200


In [411]:
# with open("ML_sts.pickle", "wb") as f:
#     pickle.dump(best_model, f)